# 모델 성능 비교

동일한 문서에 대해 여러개의 테스트 케이스(해당 문서에 관련한 질문)를 집어넣고 유사도 계산 때려보면 어떨지  
이걸 응용해서 n개의 케이스를 평균때려 계산해서 써도 될듯??
https://github.com/BM-K/Sentence-Embedding-is-all-you-need

## base model

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer

def cal_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

In [50]:
model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta-multitask')  
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta-multitask')  

sentences = ['국민의 취업준비를 위한 국민내일배움카드제가 개설되었다.', # 기존 문서
             '국민내일배움카드제가 국민의 취준을 돕고 있다.', # 검색 쿼리
             '나는 밥을 먹었다.'] # 관련없는 쿼리

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
embeddings, _ = model(**inputs, return_dict=False)

score01 = cal_score(embeddings[0][0], embeddings[1][0])  # 87.2708
# '국민의 취업준비를 위한 국민내일배움카드제가 개설되었다.' @ '국민내일배움카드제가 국민의 취준을 돕고 있다.'
score02 = cal_score(embeddings[0][0], embeddings[2][0])  # 13.7799
# '국민의 취업준비를 위한 국민내일배움카드제가 개설되었다.' @ '나는 밥을 먹었다.'

In [51]:
score01

tensor([[87.2708]], grad_fn=<MulBackward0>)

In [52]:
score02

tensor([[13.7799]], grad_fn=<MulBackward0>)

## finetuned model

In [53]:
PATH = 'your_model_path'
model = AutoModel.from_pretrained(PATH)  
tokenizer = AutoTokenizer.from_pretrained(PATH)  

In [54]:
sentences = ['국민의 취업준비를 위한 국민내일배움카드제가 개설되었다.',
             '국민내일배움카드제가 국민의 취준을 돕고 있다.',
             '나는 밥을 먹었다.']

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
embeddings, _ = model(**inputs, return_dict=False)

score01 = cal_score(embeddings[0][0], embeddings[1][0])  # 90.5397
# '국민의 취업준비를 위한 국민내일배움카드제가 개설되었다.' @ '국민내일배움카드제가 국민의 취준을 돕고 있다.'
score02 = cal_score(embeddings[0][0], embeddings[2][0])  # -1.1663
# '국민의 취업준비를 위한 국민내일배움카드제가 개설되었다.' @ '나는 밥을 먹었다.'

In [55]:
score01

tensor([[90.5397]], grad_fn=<MulBackward0>)

In [56]:
score02

tensor([[-1.1663]], grad_fn=<MulBackward0>)

In [65]:
sentences = ['20대',
             '18세에서 34세',
             '나는 밥을 먹었다.']

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
embeddings, _ = model(**inputs, return_dict=False)

score01 = cal_score(embeddings[0][0], embeddings[1][0])  # 90.5397
# '국민의 취업준비를 위한 국민내일배움카드제가 개설되었다.' @ '국민내일배움카드제가 국민의 취준을 돕고 있다.'
score02 = cal_score(embeddings[0][0], embeddings[2][0])  # -1.1663
# '국민의 취업준비를 위한 국민내일배움카드제가 개설되었다.' @ '나는 밥을 먹었다.'

In [66]:
score01

tensor([[46.0533]], grad_fn=<MulBackward0>)